<a href="https://colab.research.google.com/github/lucarubini/LINKS_DeepLearning_Course/blob/main/scripts/pytorch/02_pytorch_tutorial_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optimizing Models parameters

*  we have a model and data it’s time to train, validate and test our model
*  optimizing its parameters on our data.
*  Training a model is an iterative process:
    1.  model makes a guess about the output
    2.  calculates the error in its guess (loss) collects the derivatives of the error with respect to its parameters (as we saw in the previous section)
    3. optimize these parameters using gradient descent.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


**Hyperparameters** are adjustable parameters that let you control the model optimization process.

Different hyperparameter values can impact model training and convergence rates (read more about hyperparameter tuning)

These are the most important parameters for training:
*  **Number of Epochs**: the number times to iterate over the dataset
*  **Batch Size**: the number of data samples propagated through the network before the parameters are updated
*  **Learning Rate**: how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Optimization is the process of adjusting model parameters to reduce model error  n each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent).

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the **train_loop**, optimization happens in three steps:
1.  Call optimizer.zero_grad() to reset the gradients of model parameters.
2.  Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
3. Backpropagate the prediction loss with a call to loss.backwards(). (*PyTorch deposits the gradients of the loss w.r.t. each parameter.*)

Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [6]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314284  [    0/60000]
loss: 2.304525  [ 6400/60000]
loss: 2.277169  [12800/60000]
loss: 2.265716  [19200/60000]
loss: 2.259385  [25600/60000]
loss: 2.224501  [32000/60000]
loss: 2.226746  [38400/60000]
loss: 2.192831  [44800/60000]
loss: 2.195811  [51200/60000]
loss: 2.160571  [57600/60000]
Test Error: 
 Accuracy: 48.6%, Avg loss: 2.159291 

Epoch 2
-------------------------------
loss: 2.169537  [    0/60000]
loss: 2.165239  [ 6400/60000]
loss: 2.104408  [12800/60000]
loss: 2.116774  [19200/60000]
loss: 2.078317  [25600/60000]
loss: 2.008317  [32000/60000]
loss: 2.035556  [38400/60000]
loss: 1.952197  [44800/60000]
loss: 1.957154  [51200/60000]
loss: 1.892005  [57600/60000]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.888249 

Epoch 3
-------------------------------
loss: 1.917294  [    0/60000]
loss: 1.894153  [ 6400/60000]
loss: 1.772933  [12800/60000]
loss: 1.811970  [19200/60000]
loss: 1.713124  [25600/60000]
loss: 1.653918  [32000/600